## Exploring the data
Since there is so much data, we need to figure out what the data is and how, if we want to, to combine all the data. We also need to check if anything needs to be cleaned. 

In [131]:
import pandas as pd 
import numpy as np
import sqlite3

In [132]:
df_gross = pd.read_csv('../data/bom.movie_gross.csv', index_col=0)
df_budgets = pd.read_csv('../data/tn.movie_budgets.csv', index_col=0)
df_movies = pd.read_csv('../data/tmdb.movies.csv', index_col=0)
df_reviews = pd.read_csv('../data/rt.reviews.tsv', index_col=0, sep='\t', encoding='latin-1')
df_info = pd.read_csv('../data/rt.movie_info.tsv', index_col=0, sep='\t')
conn = sqlite3.connect('../data/im.db')

This dataset holds the amount of money a movie made domestically and foreignly, the studio associated with the movie and the year it came out.

In [133]:
#df_gross['domestic_gross'] = df_gross['domestic_gross'].map(lambda x: int(x) if pd.notnull(x) else x) Not working??
df_gross

,studio,domestic_gross,foreign_gross,year
title,,,,
Toy Story 3,BV,415000000.0,652000000,2010
Alice in Wonderland (2010),BV,334200000.0,691300000,2010
Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
Inception,WB,292600000.0,535700000,2010
Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...
The Quake,Magn.,6200.0,NaN,2018
Edward II (2018 re-release),FM,4800.0,NaN,2018
El Pacto,Sony,2500.0,NaN,2018


This dataset is somewhat similar to the one above, except it has a more specific release date, and also the budget spent on the movie. Upon checking the gross columns, there are no missing boxes, so we can strip the columns and turn them into ints to make them easier to deal with. We will also drop movies before the 2000s as inflation will make the price comparisons not equal.

In [134]:
#df_budgets[['production_budget', 'domestic_gross', 'wordlwide_gross']] = df_budgets[['production_budget', 'domestic_gross', 'worldwide_gross']].apply(lambda x: x.replace('$', '').replace(',', ''))
#Remove all symbols and convert to int
df_budgets['production_budget'] = df_budgets['production_budget'].map(lambda x: int(x.replace('$', '').replace(',', '')))
df_budgets['domestic_gross'] = df_budgets['domestic_gross'].map(lambda x: int(x.replace('$', '').replace(',', '')))
df_budgets['worldwide_gross'] = df_budgets['worldwide_gross'].map(lambda x: int(x.replace('$', '').replace(',', '')))
#convert to datetime, remove everything before 2000s
df_budgets['release_date'] = pd.to_datetime(df_budgets['release_date']).dt.date
df_budgets = df_budgets[df_budgets['release_date'] > pd.to_datetime('1999-12-31')]
df_budgets

c:\Users\Eddie\anaconda3\envs\learn-env\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,2009-12-18,Avatar,425000000,760507625,2776345279
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...
77,2004-12-31,The Mongol King,7000,900,900
78,2018-12-31,Red 11,7000,0,0
80,2005-07-13,Return to the Land of Wonders,5000,1338,1338


What we can do with this dataset is create a value that measures how much the movie made compared to its production budget to get a simplified value of return.

In [135]:
df_budgets['return_ratio'] = ((df_budgets['worldwide_gross'] - df_budgets['production_budget']) / df_budgets['production_budget'] ).round(2)
df_budgets.sort_values('return_ratio', ascending=False)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,return_ratio
id,,,,,,
93,2009-09-25,Paranormal Activity,450000,107918810,194183034,430.52
80,2015-07-10,The Gallows,100000,22764410,41656474,415.56
10,2004-05-07,Super Size Me,65000,11529368,22233808,341.06
82,2005-08-05,My Date With Drew,1100,181041,181041,163.58
57,2007-05-16,Once,150000,9445857,23323631,154.49
...,...,...,...,...,...,...
4,2011-12-31,Tracker,6500000,0,3149,-1.00
81,2016-10-16,Mi America,2100000,3330,3330,-1.00
83,2012-12-31,Infected,2100000,0,0,-1.00


This dataset has an interesting column genre_ids, which holds arrays of numbers. These numbers presumably can be associated with a dict of some sort that holds what genre it is from the number. It also has a popularity number which isn't obvious what it is, the vote_average, which is presumably out of 10, and the vote count. We will drop the genre_ids since there doesn't seem to be another table that links it to the actual genres.

In [136]:
df_movies.drop(columns='genre_ids', inplace=True)
df_movies.set_index('id', inplace=True)

df_movies['release_date'] = pd.to_datetime(df_movies['release_date']).dt.date
df_movies = df_movies[df_movies['release_date'] > pd.to_datetime('1999-12-31')]
df_movies

c:\Users\Eddie\anaconda3\envs\learn-env\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,original_language,original_title,popularity,release_date,title,vote_average,vote_count
id,,,,,,,
12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229
...,...,...,...,...,...,...,...
488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1


This dataset holds a large amount of reviews with the text, and the score associated with it. This one seems to have many missing rating numbers, some not even being numbers. The fresh section shows this came from rotten tomatoes.

In [137]:
df_reviews

,review,rating,fresh,critic,top_critic,publisher,date
id,,,,,,,
3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...
2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"


This dataset has a lot of info in it. The genre, director, writer, dates, runtime, studio, to name a few which might not be found in the other datasets.

In [138]:
df_info

,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
id,,,,,,,,,,,
1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN


The rotten tomato datasets may be one we decide to not use. Not only are the columns missing a lot of values, neither datasets have titles that we can tie the data to. Another option would be to instead just atribute the data to studios, but even the studio data is pretty sparse and missing a lot of data.

We need to figure out the structure of this database first. There are quite a few tables and the key that connects them is unknown. There might be data that isn't shown in the other datasets, such as principals, movie_akas, and known_for. So for now, we will explore those since the other tables likely have redundant data.

In [139]:
pd.read_sql(
"""
SELECT name
FROM sqlite_master
WHERE type='table'            
""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


It seems like in these databases, people are not referred to by name, instead by an id that will link them to another table. In any case, this table shows the people who worked on a movie, their roles, and potentially the characters name they played.

In [140]:
pd.read_sql(
"""
SELECT *
FROM principals
""", conn)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


This table seems to be for movies that have different titles since they're in a different language. 

In [141]:
pd.read_sql(
"""
SELECT *
FROM movie_akas
""", conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


This table relates a person to the movies they worked on.

In [142]:
pd.read_sql(
"""
SELECT *
FROM known_for
""",conn)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
...,...,...
1638255,nm9990690,tt9090932
1638256,nm9990690,tt8737130
1638257,nm9991320,tt8734436
1638258,nm9991320,tt9615610


With some basic cleaning done, and a little quick analysis, we can begin doing our own separate analysis and creating business recommendations

In [143]:
df_gross.to_csv('../data/cleaned_movie_gross.csv', encoding='utf-8')
df_budgets.to_csv('../data/cleaned_budgets.csv', encoding='utf-8')
df_movies.to_csv('../data/cleaned_movies.csv', encoding='utf-8')